<a href="https://colab.research.google.com/github/sreehari59/Polar-Embedding/blob/main/Polar_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gensim
from numpy import linalg
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath

# This is for the plotly subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# This is to load file from google drive
from google.colab import drive
drive.mount('/content/drive')

# This is to download csv file
from google.colab import files

import random
import scipy


!pip install ipython-autotime
%load_ext autotime

Mounted at /content/drive
time: 5.31 ms (started: 2022-05-11 23:01:58 +00:00)


In [2]:
def unzipping_file():
  !wget https://nlp.stanford.edu/data/glove.6B.zip
  !unzip glove*.zip # When it is unzipped you can see the different set of dimension it can have that is 25d, 50d, 100d or 200d

time: 3.11 ms (started: 2022-05-11 23:01:58 +00:00)


In [3]:
def model_initialization(unzippedname, new_name):
  model_glove = glove2word2vec(unzippedname,new_name)
  model_glove = gensim.models.KeyedVectors.load_word2vec_format(new_name, binary=False)
  return model_glove

time: 4.71 ms (started: 2022-05-11 23:01:58 +00:00)


In [4]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))  
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    
    temp_file.close()
    return output_path

time: 10.8 ms (started: 2022-05-11 23:01:58 +00:00)


In [5]:
def load_normalized_embedding(output_file_name):
  return gensim.models.KeyedVectors.load_word2vec_format(output_file_name,binary=True)

time: 1.77 ms (started: 2022-05-11 23:01:58 +00:00)


**Loading the Antonym**

In [6]:
def load_antonyms(list_new,current_model):
  list_new= list(dict.fromkeys(list_new).keys())

  similarity_matrix = defaultdict(list)
  for each_pair in tqdm(list_new):
      word1 = each_pair[0]
      word2 = each_pair[1]
      if word1 < word2:
          similarity_matrix[word1].append(word2)
      else:
          similarity_matrix[word2].append(word1)

  all_similarity = defaultdict(dict)
  for each_key in tqdm(similarity_matrix):
      for each_value in similarity_matrix[each_key]:
  #         cosine_similarity([current_model[each_key]]
          try:
            all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
          except:
            print("The word is not in the vocabulary : ",each_key," ",each_value)

  final_list = []
  for index_counter, each_key in enumerate(tqdm(all_similarity)):
  #     print(each_key,all_similarity[each_key])
      listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
  #     print(listofTuples)
      final_list.append((each_key, listofTuples[0][0]))
  print(len(final_list))

  list_antonym = final_list
  
  return list_antonym

time: 38.1 ms (started: 2022-05-11 23:01:58 +00:00)


**Find the antonym difference vectors**

In [7]:
def antonym_difference_vector(list_antonym,current_model):
  num_antonym  = len(list_antonym)
  antonymy_vector = []
  for each_word_pair in list_antonym:
      antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
      
  antonymy_vector = np.array(antonymy_vector)
  print(antonymy_vector.shape)
  print(antonymy_vector)
  return antonymy_vector

time: 7.04 ms (started: 2022-05-11 23:01:58 +00:00)


In [8]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim,antonymy_vector):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    new_embedding_dict = transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

    return new_embedding_dict

time: 4.19 ms (started: 2022-05-11 23:01:58 +00:00)


In [9]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):

    temp_dict = dict()

    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    #print('New model size is',len(current_model), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)
        temp_dict[each_word] = new_vector
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()
    return temp_dict

time: 29.8 ms (started: 2022-05-11 23:01:58 +00:00)


In [10]:
def make_polar_dict(company_name, antonym, embedding, top_n = False, n = 10):
  temp_dict = dict()
  temp_polar = embedding[company_name]

  if top_n:
    idx = np.argsort([abs(x) for x in temp_polar])[-n:]
    for i in idx:
      print(antonym[i],temp_polar[i],'\n')


  if len(antonym) == len(temp_polar):
    for a in range(len(antonym)):
      temp_dict[antonym[a]] = temp_polar[a]
    return temp_dict

time: 8.68 ms (started: 2022-05-11 23:01:58 +00:00)


In [11]:
def download_csv_file(df,filename):
  df.to_csv(filename)
  files.download(filename)

time: 1.79 ms (started: 2022-05-11 23:01:58 +00:00)


**The below function will input the antonym pair from the user**

In [ ]:
def user_antonym_input(list_new,user_list):
  list_new.append(user_list)
  return list_new

**The below function will find words simliar to the given antonym pair**

In [47]:
def similar_antonym_pair(list_new,antonym_pair,current_model):
  first_word = antonym_pair.split("-")[0]
  second_word = antonym_pair.split("-")[1]

  similar_word_list1 = current_model.similar_by_word(first_word)
  similar_word_list2 = current_model.similar_by_word(second_word)
  for i in range(0,5):
    
    new_antonym_word1 = similar_word_list1[i][0]
    new_antonym_word2 = similar_word_list2[i][0]   
   
    antonym_tuple = (new_antonym_word1,new_antonym_word2)
    
    list_new.append(antonym_tuple)
    clear_tuple(antonym_tuple)
    
  return list_new

time: 12.7 ms (started: 2022-05-11 23:38:36 +00:00)


In [35]:
def clear_tuple(tuples):
  temp = list(tuples)
  temp.clear()
  tuples = tuple(temp)
  return tuples

time: 2 ms (started: 2022-05-11 23:35:06 +00:00)


In [12]:
unzipping_file()
model_glove = model_initialization('glove.6B.300d.txt','gensim_glove.6B.300d.txt')
path = generate_norm_embedding(model_glove,'glove_norm_wiki.mod')
current_model  = load_normalized_embedding(path)

--2022-05-11 23:01:58--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-05-11 23:01:58--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.18MB/s    in 2m 39s  

2022-05-11 23:04:38 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/400000 [00:00<?, ?it/s]

time: 4min 26s (started: 2022-05-11 23:01:58 +00:00)


In [13]:
list_new= [('product', 'service'),
('essential', 'luxury'),
('technical', 'natural'),
('renewable', 'nonrenewable'),
('advertising', 'rumour'), 
('lease', 'sell'), 
('tangible', 'intangible'), 
('demand', 'supply'),
('child', 'childless'),
('remote', 'physical')]

time: 5.48 ms (started: 2022-05-11 23:06:25 +00:00)


In [14]:
list_antonym  = load_antonyms(list_new,current_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/10 [00:00<?, ?it/s]

10
time: 193 ms (started: 2022-05-11 23:06:25 +00:00)


In [15]:
antonymy_vector = antonym_difference_vector(list_antonym,current_model)
print(antonymy_vector)

(10, 300)
[[ 0.12007222  0.18291782 -0.01045436 ...  0.00340314 -0.04040879
   0.04456742]
 [-0.05503665 -0.09060876 -0.00170645 ... -0.10382904 -0.16571268
  -0.05653016]
 [-0.0209384   0.0909104  -0.00575457 ... -0.02009975  0.02040304
   0.17290717]
 ...
 [ 0.01087348 -0.01034411 -0.03145081 ...  0.03090873  0.03611581
  -0.02737074]
 [-0.00376742 -0.00955552 -0.10660151 ... -0.01117709 -0.08223383
  -0.01124656]
 [ 0.08622979  0.03551516 -0.04093393 ... -0.00040361  0.0736662
  -0.09048237]]
[[ 0.12007222  0.18291782 -0.01045436 ...  0.00340314 -0.04040879
   0.04456742]
 [-0.05503665 -0.09060876 -0.00170645 ... -0.10382904 -0.16571268
  -0.05653016]
 [-0.0209384   0.0909104  -0.00575457 ... -0.02009975  0.02040304
   0.17290717]
 ...
 [ 0.01087348 -0.01034411 -0.03145081 ...  0.03090873  0.03611581
  -0.02737074]
 [-0.00376742 -0.00955552 -0.10660151 ... -0.01117709 -0.08223383
  -0.01124656]
 [ 0.08622979  0.03551516 -0.04093393 ... -0.00040361  0.0736662
  -0.09048237]]
time: 7 

In [16]:
company = pd.read_csv('/content/drive/MyDrive/International_Fortune_GloVe.csv',encoding= 'unicode_escape')
name_list = company['0']

time: 305 ms (started: 2022-05-11 23:06:25 +00:00)


In [17]:
name_word_embedding = dict()
new_name_list=[]
for name in name_list:
  try:
    name_word_embedding[name] = current_model[name]
    new_name_list.append(name)
  except:
    print("The company is not there: ",name)

print(new_name_list)

The company is not there:  jpmorganchase
The company is not there:  generalmotors
The company is not there:  fanniemae
The company is not there:  bankofamerica
The company is not there:  nestlÃ©
The company is not there:  wellsfargo
The company is not there:  generalelectric
The company is not there:  sinopharm
The company is not there:  freddiemac
The company is not there:  waltdisney
The company is not there:  engie
The company is not there:  morganstanley
The company is not there:  indianoil
The company is not there:  telefÃ³nica
The company is not there:  bestbuy
The company is not there:  talanx
The company is not there:  abbvie
The company is not there:  americanexpress
The company is not there:  deutschebank
The company is not there:  tatamotors
The company is not there:  l'orÃ©al
The company is not there:  angloamerican
The company is not there:  achmea
The company is not there:  dollartree
The company is not there:  riteaid
['walmart', 'amazon', 'apple', 'volkswagen', 'mckesso

In [18]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))
name_new_embedding = generate_embedding_path(name_word_embedding, 'test_run',True ,random_antonym_vector,500,antonymy_vector)

10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/142 [00:00<?, ?it/s]

time: 102 ms (started: 2022-05-11 23:06:26 +00:00)


In [19]:
antonym_500 = [list_antonym[x] for x in random_antonym_vector[:500]]

time: 1.82 ms (started: 2022-05-11 23:06:26 +00:00)


**Polar Embedding -> DataFrame**

In [20]:
df = dict()
for t in new_name_list:
  df[t] = make_polar_dict(t, antonym_500, name_new_embedding)

new_df = pd.DataFrame(df).transpose()


# change columns to better read names
new_columns = []

for pair in antonym_500:
  temp = pair[0]+''+pair[1]
  new_columns.append(temp)

new_df.columns = new_columns

time: 62.3 ms (started: 2022-05-11 23:06:26 +00:00)


In [21]:
new_df.head()

,essentialluxury,naturaltechnical,advertisingrumour,childchildless,productservice,demandsupply,intangibletangible,leasesell,nonrenewablerenewable,physicalremote
walmart,-0.397833,0.547627,0.053615,-0.234927,0.210923,-0.610311,-0.107264,-0.203712,0.094820,-0.069312
amazon,0.149086,0.337541,0.274870,0.256857,0.154932,0.244374,-0.073648,-0.074484,0.017427,-0.792030
apple,0.019489,-0.241936,0.276218,0.059273,0.446798,0.053345,-0.029875,-0.630734,-0.369896,-0.351087
volkswagen,-0.680670,-0.189240,0.166854,-0.164525,0.358346,-0.171160,-0.164260,-0.273070,-0.426517,0.069241
mckesson,-0.033068,0.196849,0.185271,-0.725982,-0.037312,-0.436756,0.277886,-0.190578,-0.052383,0.300580


time: 47.1 ms (started: 2022-05-11 23:06:26 +00:00)


In [22]:
fortune_500_company = pd.read_csv('/content/drive/MyDrive/Fortune Global 500 companies.csv',encoding= 'unicode_escape')
fortune_500_company['Company'] = fortune_500_company['Company'].str.lower()
fortune_500_company['Company'] = fortune_500_company['Company'].str.replace(" ","")


time: 388 ms (started: 2022-05-11 23:06:26 +00:00)


In [51]:
polar_embedding = pd.merge(fortune_500_company,new_df,how="right",left_on="Company",right_on=new_df.index)

,Rank,Company,Location
0,1,walmart,USA
1,2,stategrid,China
2,3,amazon,USA
3,4,chinanationalpetroleum,China
4,5,sinopecgroup,China


time: 47.5 ms (started: 2022-05-11 23:52:11 +00:00)


In [62]:
polar_embedding.drop(['Rank'],axis=1)

,Company,Location,essentialluxury,naturaltechnical,advertisingrumour,childchildless,productservice,demandsupply,intangibletangible,leasesell,nonrenewablerenewable,physicalremote
0,walmart,USA,-0.397833,0.547627,0.053615,-0.234927,0.210923,-0.610311,-0.107264,-0.203712,0.094820,-0.069312
1,amazon,USA,0.149086,0.337541,0.274870,0.256857,0.154932,0.244374,-0.073648,-0.074484,0.017427,-0.792030
2,apple,USA,0.019489,-0.241936,0.276218,0.059273,0.446798,0.053345,-0.029875,-0.630734,-0.369896,-0.351087
3,volkswagen,Germany,-0.680670,-0.189240,0.166854,-0.164525,0.358346,-0.171160,-0.164260,-0.273070,-0.426517,0.069241
4,mckesson,USA,-0.033068,0.196849,0.185271,-0.725982,-0.037312,-0.436756,0.277886,-0.190578,-0.052383,0.300580
...,...,...,...,...,...,...,...,...,...,...,...,...
137,netflix,USA,-0.036024,0.295033,0.232239,-0.497865,-0.321629,0.345320,-0.564339,0.000327,-0.014090,-0.261697
138,nokia,Finland,-0.242810,-0.502687,0.292643,-0.113547,0.587839,0.119778,0.123189,-0.035180,-0.336982,-0.316139
139,synnex,USA,-0.095035,-0.180539,-0.118953,-0.237965,0.678891,0.005976,0.274964,0.164871,0.522543,0.225455
140,holcim,Switzerland,-0.394086,0.267346,-0.026292,-0.085538,0.727239,-0.287880,0.131076,-0.239961,0.146410,-0.239300


time: 65.2 ms (started: 2022-05-11 23:57:18 +00:00)


In [64]:
# This will find the total number of companies in our data frame based on Location
total_company_list_based_on_loc = polar_embedding['Location'].value_counts()
total_company_count_df = pd.DataFrame({'Country':total_company_list_based_on_loc.index, 'Total Count':total_company_list_based_on_loc.values})
print(total_company_count_df.head())


   Country  Total Count
0      USA           52
1    Japan           18
2  Germany           12
3  Britain           10
4   France            9
time: 13.7 ms (started: 2022-05-12 00:14:24 +00:00)


In [70]:
input_list = ['essential-luxury',	'natural-technical'	,'advertising-rumour',	'child-childless',	'product-service',	'demand-supply',	'intangible-tangible'	,
 'lease-sell'	,'nonrenewable-renewable',	'physical-remote']

company_df=total_company_count_df.copy()

# we then find the number of companies grouped on the basis of location
for i in input_list:
  print(i)
  j = i.replace("-","")
  subset_df2 = polar_embedding[polar_embedding[j] < 0]
  company_inclined_to_left_polar_df1 = subset_df2['Location'].value_counts()
  left_polar = i.split("-")[0]
  company_inclined_to_left_polar_df1 = pd.DataFrame({'Country':company_inclined_to_left_polar_df1.index, left_polar :company_inclined_to_left_polar_df1.values})
  company_df=pd.merge(company_df, company_inclined_to_left_polar_df1, how='left',on='Country')    
  company_df[left_polar] = round( company_df[left_polar] / company_df.iloc[:,1] * 100)

  subset_df1 = polar_embedding[polar_embedding[j] > 0]
  company_inclined_to_right_polar_df1 = subset_df1['Location'].value_counts()
  right_polar = i.split("-")[1]
  company_inclined_to_right_polar_df1 = pd.DataFrame({'Country':company_inclined_to_right_polar_df1.index, right_polar :company_inclined_to_right_polar_df1.values})
  company_df=pd.merge(company_df, company_inclined_to_right_polar_df1, how='left',on='Country')    
  company_df[right_polar] = round( company_df[right_polar] / company_df.iloc[:,1] * 100)


company_df = company_df.fillna(0)

# We are considering only the countries if the numberof companies in the country is over 3
company_df = company_df[company_df['Total Count'] > 3]
print(company_df.head())
print(company_df.shape)

essential-luxury
natural-technical
advertising-rumour
child-childless
product-service
demand-supply
intangible-tangible
lease-sell
nonrenewable-renewable
physical-remote
   Country  Total Count  essential  luxury  natural  technical  advertising  \
0      USA           52       63.0    37.0     44.0       56.0         31.0   
1    Japan           18       89.0    11.0     72.0       28.0         44.0   
2  Germany           12       92.0     8.0     58.0       42.0         33.0   
3  Britain           10       90.0    10.0     20.0       80.0         40.0   
4   France            9      100.0     0.0     33.0       67.0         67.0   

   rumour  child  childless  ...  demand  supply  intangible  tangible  lease  \
0    69.0   79.0       21.0  ...    54.0    46.0        50.0      50.0   63.0   
1    56.0   78.0       22.0  ...    44.0    56.0        11.0      89.0   67.0   
2    67.0   83.0       17.0  ...   100.0     0.0        58.0      42.0  100.0   
3    60.0  100.0        0.0  ..

In [76]:
polar_list = ["essential",'natural','advertising','child','physical']

# The user input the country name and get the graph for different synonyms under a single Hofstede's Dimension
total_sum=0
total_sum_list=[]
polar_ranking_list = []
polar_index=0
for index,row in company_df.iterrows():  
  
  for i in polar_list:
    
    total_sum = total_sum + (row[i])
  #print(company_df.iloc[index,2:])  
  total_sum_list.append(total_sum/len(polar_list))
  polar_ranking_list.append(index+1)
  total_sum = 0

company_df['Total Score'] = total_sum_list
company_df= company_df.sort_values(by=['Total Score'],ascending=False)
company_df['Polar Ranking'] = polar_ranking_list

time: 24 ms (started: 2022-05-12 00:29:52 +00:00)


In [77]:
company_df.head()

,Country,Total Count,essential,luxury,natural,technical,advertising,rumour,child,childless,...,intangible,tangible,lease,sell,nonrenewable,renewable,physical,remote,Total Score,Polar Ranking
6,South Korea,4,75.0,25.0,75.0,25.0,100.0,0.0,75.0,25.0,...,0.0,100.0,75.0,25.0,25.0,75.0,50.0,50.0,75.0,1
1,Japan,18,89.0,11.0,72.0,28.0,44.0,56.0,78.0,22.0,...,11.0,89.0,67.0,33.0,67.0,33.0,61.0,39.0,68.8,2
4,France,9,100.0,0.0,33.0,67.0,67.0,33.0,78.0,22.0,...,78.0,22.0,78.0,22.0,67.0,33.0,33.0,67.0,62.2,3
5,Switzerland,5,100.0,0.0,20.0,80.0,80.0,20.0,80.0,20.0,...,20.0,80.0,60.0,40.0,40.0,60.0,20.0,80.0,60.0,4
3,Britain,10,90.0,10.0,20.0,80.0,40.0,60.0,100.0,0.0,...,30.0,70.0,60.0,40.0,80.0,20.0,40.0,60.0,58.0,5


time: 88.4 ms (started: 2022-05-12 00:29:54 +00:00)


In [ ]:
hofstede_df = pd.read_csv("/content/drive/MyDrive/Polar Embedding/Hofstede 6 dimensions.csv",sep=";")
hofstede_df=hofstede_df[hofstede_df.iloc[:,:]!="#NULL!" ]
hofstede_df.dropna(axis=0)

In [ ]:
def percentage_difference(hofstede_dimension):

  for index,row in hofstede_df.iterrows():
    if row['Country'] == country_name:
      hofstede_dimension_value = int(row[hofstede_dimension])
    elif row['ctr'] == country_name:
      hofstede_dimension_value = int(row[hofstede_dimension])
  
  mean_value = total_sum/len(polar_list)
  if hofstede_dimension_value > mean_value:
    print("% difference",hofstede_dimension_value - mean_value )
  else:
    print("% difference",mean_value - hofstede_dimension_value )